#1. Library imports

In [1]:
!pip install -q transformers
!pip install -q datasets

     |████████████████████████████████| 3.8 MB 10.6 MB/s 
     |████████████████████████████████| 6.5 MB 41.1 MB/s 
     |████████████████████████████████| 596 kB 44.0 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 895 kB 43.4 MB/s 
     |████████████████████████████████| 325 kB 12.4 MB/s 
     |████████████████████████████████| 134 kB 36.2 MB/s 
     |████████████████████████████████| 212 kB 35.3 MB/s 
     |████████████████████████████████| 1.1 MB 10.3 MB/s 
     |████████████████████████████████| 127 kB 39.6 MB/s 
     |████████████████████████████████| 271 kB 37.5 MB/s 
     |████████████████████████████████| 144 kB 33.9 MB/s 
     |████████████████████████████████| 94 kB 1.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which 

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import tqdm
import nltk
import torch
import numpy as np
import pandas as pd
from torch import nn
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import confusion_matrix, classification_report
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import transformers

In [5]:
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import tqdm
from sklearn.metrics import confusion_matrix, classification_report

In [6]:
import tempfile
import pathlib
import pyarrow as pa
import pyarrow.parquet as pq

# 2. Data Imports

In [7]:
def load_data(split_name='train', columns=['text', 'stars'], folder='gdrive/MyDrive/COMP_4332/Project1/data'):
        df = pd.read_csv(f'{folder}/{split_name}.csv')
        return df

In [8]:
train_df = load_data('train', columns=['text', 'stars'])
valid_df = load_data('valid', columns=['text', 'stars'])
# the test set labels (the 'stars' column) are not available! So the following code will instead return all columns
test_df = load_data('test', columns=['text', 'stars'])

In [9]:
train_df = train_df[["text","stars"]]
valid_df = valid_df[["text","stars"]]

# MODEL - BERT using TF

## Tokenizer

In [10]:
import datasets
import pandas as pd
import tensorflow as tf
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
def tokenize_function(x):
    return tokenizer(x, truncation=True)#,padding="max_length",max_length=512)

def tokenize_df(df):
  tokens = df['text'].map(tokenize_function)
  df['input_ids'] = [x['input_ids'] for x in tokens]
  df['attention_mask'] = [x['attention_mask'] for x in tokens]
  df['token_type_ids'] = [x['token_type_ids'] for x in tokens]
  df.rename(columns={"stars":"labels"},inplace=True)
  return df

train_df = tokenize_df(train_df)
valid_df = tokenize_df(valid_df)
test_df = tokenize_df(test_df)

In [12]:
print(train_df.columns)
print(valid_df.columns)
print(test_df.columns)

Index(['text', 'labels', 'input_ids', 'attention_mask', 'token_type_ids'], dtype='object')
Index(['text', 'labels', 'input_ids', 'attention_mask', 'token_type_ids'], dtype='object')
Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'text', 'useful',
       'user_id', 'input_ids', 'attention_mask', 'token_type_ids'],
      dtype='object')


In [13]:
small_train = train_df.head(3000)
small_validate = valid_df.head(1000)

In [14]:
table_train = pa.table({'labels': (list(small_train["labels"])), 'input_ids':list(small_train["input_ids"]), 'attention_mask':list(small_train["attention_mask"]),'token_type_ids':list(small_train["token_type_ids"]) })
table_validate = pa.table({'labels': list(small_validate["labels"]),'input_ids':list(small_validate["input_ids"]), 'attention_mask':list(small_validate["attention_mask"]),'token_type_ids':list(small_validate["token_type_ids"])})

In [15]:
#training = datasets.DatasetDict({"labels":list(train_df["labels"]), "text": list(train_df["text"])})
training = datasets.Dataset(table_train)
valid = datasets.Dataset(table_validate)

In [16]:
training.features
valid.features

{'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'labels': Value(dtype='int64', id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [17]:
MyDataset = datasets.DatasetDict({"train":training,"validation":valid})

# Loading Data

In [18]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [23]:
tf_train_dataset = MyDataset["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = MyDataset["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

# Model - sequence

In [37]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    MyDataset["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
valid_dataloader = DataLoader(
    MyDataset["validation"], batch_size=8, collate_fn=data_collator
)

In [40]:
from tqdm.auto import tqdm
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/1125 [00:00<?, ?it/s]

TypeError: ignored

# Train

In [33]:
from transformers import AdamW,get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

750


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
